In [ ]:
import os
from pathlib import Path
project_root = os.path.join(str(Path.home()), 'diffusion_models')
os.chdir(project_root)
%pwd # should be PPGA root dir

In [ ]:
import pickle
import torch
import numpy as np

from autoencoders.policy.resnet3d import ResNet3DAutoEncoder
from autoencoders.policy.hypernet import HypernetAutoEncoder
from attrdict import AttrDict
from RL.actor_critic import Actor
from envs.brax_custom.brax_env import make_vec_env_brax
from IPython.display import HTML, Image
from IPython.display import display
from brax.io import html, image
from dataset.tensor_elites_dataset import preprocess_model, postprocess_model
from utils.brax_utils import shared_params
from tqdm import tqdm

In [ ]:
# params to config
device = torch.device('cuda')
env_name = 'walker2d'
seed = 1111
normalize_obs = True
normalize_rewards = True
obs_shape = shared_params[env_name]['obs_dim']
action_shape = shared_params[env_name]['action_dim']
mlp_shape = (128, 128, action_shape)

env_cfg = AttrDict({
    'env_name': env_name,
    'env_batch_size': None,
    'num_dims': 2,
    'seed': seed,
    'num_envs': 1
})

In [ ]:
archive_df_path = 'data/walker2d/archive_100x100.pkl'
with open(archive_df_path, 'rb') as f:
    archive_df = pickle.load(f)

In [ ]:
# make the env
env = make_vec_env_brax(env_cfg)

In [ ]:
def integrate_obs_normalizer(agent: Actor):
    assert agent.obs_normalizer is not None
    w_in = agent.actor_mean[0].weight.data
    b_in = agent.actor_mean[0].bias.data
    mean, var = agent.obs_normalizer.obs_rms.mean, agent.obs_normalizer.obs_rms.var
    w_new = w_in / torch.sqrt(var + 1e-8)
    b_new = b_in - (mean / torch.sqrt(var + 1e-8)) @ w_in.T
    agent.actor_mean[0].weight.data = w_new
    agent.actor_mean[0].bias.data = b_new
    return agent


In [ ]:
def enjoy_brax(agent, render=True, deterministic=True, normalize_obs=False):
    if normalize_obs:
        obs_mean, obs_var = agent.obs_normalizer.obs_rms.mean, agent.obs_normalizer.obs_rms.var
        print(f'{obs_mean=}, {obs_var=}')

    obs = env.reset()
    rollout = [env.unwrapped._state]
    total_reward = 0
    measures = torch.zeros(env_cfg.num_dims).to(device)
    done = False
    while not done:
        with torch.no_grad():
            obs = obs.unsqueeze(dim=0).to(device)
            if normalize_obs:
                obs = (obs - obs_mean) / torch.sqrt(obs_var + 1e-8)

            if deterministic:
                act = agent.actor_mean(obs)
            else:
                act, _, _ = agent.get_action(obs)
            act = act.squeeze()
            obs, rew, done, info = env.step(act.cpu())
            measures += info['measures']
            rollout.append(env.unwrapped._state)
            total_reward += rew
    if render:
        i = HTML(html.render(env.unwrapped._env.sys, [s.qp for s in rollout]))
        display(i)
    print(f'{total_reward=}')
    print(f' Rollout length: {len(rollout)}')
    measures /= len(rollout)
    print(f'Measures: {measures.cpu().numpy()}')
    return total_reward.detach().cpu().numpy(), measures.cpu().numpy()

In [ ]:
elites_array = archive_df.filter(regex='solution*').to_numpy()
inferred_labels = []
label_distances = []

## Rerun next two / three cells until sufficiently labelled

In [ ]:
# Render one elite
if label_distances:
    # If we have distances, use the elite with the largest distance
    elite_index = np.argmax(label_distances)
    print("elite distance to nearest label", np.max(label_distances))
    print("current label:", inferred_labels[elite_index])
else:
    # Otherwise, use a random elite
    elite_index = np.random.randint(len(elites_array))
agent = Actor(obs_shape, action_shape, True, True).deserialize(elites_array[elite_index]).to(device)
# make sure pre and post-processing are working correctly. This should return
# the exact same agent as the previous line
# agent = postprocess_model(agent, preprocess_model(agent, mlp_shape), mlp_shape, deterministic=False).to(device)
if normalize_obs:
    agent = integrate_obs_normalizer(agent)
reward, measures = enjoy_brax(agent, render=True)

In [ ]:
# Add a new label
labels_archive_idx.append([float(reward), measures.tolist(), int(elite_index),
                          "fall forwards while keeping both feet planted"]) # REPLACE LABEL HERE
with open(f"data/{env_name}/text_labels_{len(labels_archive_idx):05d}.json", "w") as f:
    json.dump(labels_archive_idx, f, indent=True)
label_distances[elite_index] = 0.0
labels_archive_idx[-1]

In [ ]:
# Recompute inferred labels
labelled_elites = np.array([elites_array[elite_index] for (returns, measures, elite_index, label) in labels_archive_idx])
labelled_elites_labels = [label for (returns, measures, elite_index, label) in labels_archive_idx]

def nearest_labelled_elite(archive_index):
    distances = ((labelled_elites - elites_array[archive_index]) ** 2).sum(axis=-1)
    return int(np.argmin(distances)), float(distances.min())

inferred_labels = []
label_distances = []
for archive_index in tqdm(range(len(elites_array))):
    inferred_label, label_dist = nearest_labelled_elite(archive_index)
    inferred_labels.append(labelled_elites_labels[inferred_label])
    label_distances.append(label_dist)

In [ ]:
with open(f"data/{env_name}/text_labels_{len(labels_archive_idx):05d}.pkl", "wb") as f:
    pickle.dump(inferred_labels, f)

In [ ]:
import glob
sorted(glob.glob(f"data/{env_name}/text_labels_*.pkl"))

In [ ]:
labels_archive_idx[-1]